# Homework 1 - Wikipedia Web Traffic Time Series

У вас есть данные по посещению 1000 страниц  Википедии из разных стран и разных девайсов ( \*  * данные взяты из [Kaggle соревнования](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train* и *wikipedia_test* - содержат данные о трафике. Это файлы csv, где каждая строка соответствует определенной статье, и каждый столбец соответствует конкретной дате. В некоторых записях отсутствуют данные. Названия страниц содержат проект Википедии (например, en.wikipedia.org), тип доступа (например, desktop) и тип агента (например, spider). Другими словами, каждое имя статьи имеет следующий формат: «name_project_access_agent» (например, «AKB48_zh.wikipedia.org_all-access_spider»).

Вам нужно ответить на [вопросы](https://docs.google.com/forms/d/e/1FAIpQLSfDjWeeZJw5EvmKn1x_6b9xicjn7ed3MF0rbNm4Cmwr7psSkQ/viewform?usp=sf_link) и попробовать сделать самую простую модель которая сможет предсказывать будущие посещения. 

Вот примеры временных рядов посещаемости страниц Википедии (*синие* - обучающая выборка, *зеленые* - предсказания модели победителя соревнования на Kaggle, *оранжевые* - реальные значения):
![Wikipedia Web Traffic Time Series](https://image.ibb.co/cUpEJa/predictions.png)

In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("../data/wikipedia_train.csv")
test = pd.read_csv("../data/wikipedia_test.csv")

## Data Analysis

In [ ]:
def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res.group(0)[0:2]
    return 'na'

## Forecasting

Нужно преобразовать `train` данные в следующий формат:

In [14]:
train.head()

,Page,date,Visits
0,15._November_de.wikipedia.org_desktop_all-agents,2015-07-01,32
1,2012_(film)_fr.wikipedia.org_all-access_spider,2015-07-01,2
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2015-07-01,1
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2015-07-01,3
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2015-07-01,24


Таким образом у вас каждая сточка содержит набор фич (`Page`, `date`) и целевую переменную (`Visits`). Преобразовать данные в такой формат поможет функция `pd.melt()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)

Оценивать качество предсказаний мы будем с помощью [SMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error) :

In [ ]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

Что б оценить прогноз нужно представить `test` данные в следующем формате:

In [12]:
prediciton.head()

,Page,date,Visits,pred_Visits
0,15._November_de.wikipedia.org_desktop_all-agents,2016-09-10,43,37
1,2012_(film)_fr.wikipedia.org_all-access_spider,2016-09-10,6,5
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2016-09-10,4,4
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2016-09-10,15,14
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2016-09-10,101,173


И затем вызвать функцию для подсчета метрики:

In [ ]:
pandas_smape(prediciton)

### Last day baseline

Нужно сделать прогноз на основе посещений в последний известный нам день из train (продублировать значение для каждого дня в test)

### Median baseline

Нужно сделать прогноз на основе медианы за последние **30** дней из `train`. 

А затем улучшить предсказания используя информацию выходной это или нет (вам поможет функция [dayofweek](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.dayofweek.html) ) и разные окна для подсчета медианы (7 дней, 60 дней и тд)

Вам поможет функция `pd.groupby()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)